In [1]:
import numpy as np
import pandas as pd
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from tqdm import tqdm
from itertools import permutations
import seaborn as sns
import matplotlib.pyplot as plt
from Bio.Restriction import BsaI, BsmBI
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqUtils import MeltingTemp as mt
from primers import primers
import primer3 as pr
import warnings
warnings.filterwarnings("ignore")

In [2]:
#import fasta files
fastas = ['test-flo']
plasmids = pd.DataFrame([(p.id, str(p.seq)) for fasta in fastas \
                        for p in list(SeqIO.parse('datasets/jump/{}.fasta'.format(fasta), 'fasta'))], \
                        columns=['name', 'sequence'])
plasmids['sequence'] = plasmids['sequence'].str.upper()
plasmids.head()

,name,sequence
0,pLUC0080,ATCGATGCATAATGTGCCTGTCAAATGGACGAAGCAGGGATTCTGC...


In [19]:
gene = 'ATGAGCCAGATCCTGAAGACTTCCCTGCTGATCGTCGGCGGCGGTCCCGGCGGCTACGTCGCGGCGATCCGTGCCGGGCAACTGGGCATTCCCACCGTACTGGTGGAGGGCGCCGCCCTCGGCGGCACCTGCCTGAACGTCGGCTGCATCCCGTCGAAGGCGCTGATCCACGCCGCCGAGGAATACCTCAAGGCCCGCCACTATGCCAGCCGGTCGGCGCTGGGCATCCAGGTACAGGCGCCGAGCATCGACATCGCCCGCACCGTGGAATGGAAGGACGCCATCGTCGACCGCCTCACCAGCGGCGTCGCCGCGCTGCTGAAGAAACACGGGGTCGATGTCGTCCAGGGCTGGGCGAGGATCCTCGACGGCAAAAGCGTGGCGGTCGAACTCGCCGGCGGCGGCAGCCAGCGCATCGAGTGCGAGCACCTGCTGCTGGCCGCCGGCTCGCAGAGCGTCGAGCTACCGATCCTGCCGCTGGGCGGCAAGGTGATCTCCTCCACCGAGGCGCTGGCGCCCGGCAGCCTGCCCAAGCGCCTGGTGGTGGTCGGCGGCGGCTACATCGGCCTGGAGCTGGGTACCGCCTACCGCAAGCTCGGCGTCGAGGTGGCGGTGGTGGAAGCGCAACCACGCATCCTGCCGGGCTACGACGAAGAACTGACCAAGCCGGTGGCCCAGGCCTTGCGCAGGCTGGGCGTCGAGCTGTACCTCGGGCACAGCCTGCTGGGCCCGAGCGAGAACGGCGTGCGGGTCCGCGACGGCGCCGGCGAGGAGCGCGAGATCGCCGCCGACCAGGTACTGGTGGCGGTCGGCCGCAAGCCGCGCAGCGAAGGCTGGAACCTGGAAAGCCTGGGCCTGGACATGAACGGCCGGGCGGTGAAGGTCGACGACCAGTGCCGCACCTCGATGCGCAATGTCTGGGCCATAGGCGATCTCGCCGGCGAGCCGATGCTCGCGCACCGGGCCATGGCCCAGGGCGAGATGGTCGCCGAGCTGATCGCCGGCAAGCGTCGCCAGTTCGCCCCGGTGGCGATCCCCGCGGTGTGCTTCACCGATCCGGAAGTGGTGGTCGCCGGGTTGTCCCCGGAGCAGGCGAAGGATGCCGGCCTGGACTGCCTGGTGGCGAGCTTCCCGTTCGCCGCCAACGGTCGCGCCATGACCCTGGAGGCCAACGAAGGCTTCGTCCGCGTGGTGGCGCGTCGCGACAACCACCTGGTCGTCGGCTGGCAGGCGGTGGGCAAGGCGGTCTCGGAACTGTCCACGGCCTTCGCCCAGTCGCTGGAGATGGGCGCCCGCCTGGAAGACATCGCCGGCACCATCCACGCCCATCCGACCCTCGGCGAAGCGGTCCAGGAAGCCGCCCTGCGCGCGCTGGGACACGCCCTGCACATCTGA'

In [20]:
source = 'ATCGATGCATAATGTGCCTGTCAAATGGACGAAGCAGGGATTCTGCAAACCCTATGCTACTCCGTCAAGCCGTCAATTGTCTGATTCGTTACCAATTATGACAACTTGACGGCTACATCATTCACTTTTTCTTCACAACCGGCACGGAACTCGCTCGGGCTGGCCCCGGTGCATTTTTTAAATACCCGCGAGAAATAGAGTTGATCGTCAAAACCAACATTGCGACCGACGGTGGCGATAGGCATCCGGGTGGTGCTCAAAAGCAGCTTCGCCTGGCTGATACGTTGGTCCTCGCGCCAGCTTAAGACGCTAATCCCTAACTGCTGGCGGAAAAGATGTGACAGACGCGACGGCGACAAGCAAACATGCTGTGCGACGCTGGCGATATCAAAATTGCTGTCTGCCAGGTGATCGCTGATGTACTGACAAGCCTCGCGTACCCGATTATCCATCGGTGGATGGAGCGACTCGTTAATCGCTTCCATGCGCCGCAGTAACAATTGCTCAAGCAGATTTATCGCCAGCAGCTCCGAATAGCGCCCTTCCCCTTGCCCGGCGTTAATGATTTGCCCAAACAGGTCGCTGAAATGCGGCTGGTGCGCTTCATCCGGGCGAAAGAACCCCGTATTGGCAAATATTGACGGCCAGTTAAGCCATTCATGCCAGTAGGCGCGCGGACGAAAGTAAACCCACTGGTGATACCATTCGCGAGCCTCCGGATGACGACCGTAGTGATGAATCTCTCCTGGCGGGAACAGCAAAATATCACCCGGTCGGCAAACAAATTCTCGTCCCTGATTTTTCACCACCCCCTGACCGCGAATGGTGAGATTGAGAATATAACCTTTCATTCCCAGCGGTCGGTCGATAAAAAAATCGAGATAACCGTTGGCCTCAATCGGCGTTAAACCCGCCACCAGATGGGCATTAAACGAGTATCCCGGCAGCAGGGGATCATTTTGCGCTTCAGCCATACTTTTCATACTCCCGCCATTCAGAGAAGAAACCAATTGTCCATATTGCATCAGACATTGCCGTCACTGCGTCTTTTACTGGCTCTTCTCGCTAACCAAACCGGTAACCCCGCTTATTAAAAGCATTCTGTAACAAAGCGGGACCAAAGCCATGACAAAAACGCGTAACAAAAGTGTCTATAATCACGGCAGAAAAGTCCACATTGATTATTTGCACGGCGTCACACTTTGCTATGCCATAGCATTTTTATCCATAAGATTAGCGGATCCTACCTGACGCTTTTTATCGCAACTCTCTACTGTTTCTCCATACCCGTTTTTTTGGGCTAGCAGGAGGAATTCACCATGGTACCCGGGGATCCTCTAGAAATAATTTTGTTTAACTTTAAGAAGGAGATATACCATGAACACTGCCGTCGAACCCTACAAGGCTTCCTCCTTCGACCTGACCCACAAGCTCACCGTGGAAAAGCACGGGCACACCGCGCTGATCACCATCAACCACCCGCCGGCCAACACCTGGGACCGCGACTCGCTGATCGGCCTGCGCCAACTGATCGAGCACCTCAACCGCGACGACGATATCTACGCCCTGGTAGTGACCGGCCAGGGGCCGAAGTTCTTCTCCGCCGGCGCCGACCTGAACATGTTCGCCGACGGCGACAAGGCCCGCGCTCGCGAGATGGCCCGCCGCTTCGGCGAAGCCTTCGAGGCGCTGCGCGATTTCCGCGGGGTGTCGATCGCGGCGATCAACGGCTACGCCATGGGCGGCGGCCTGGAGTGCGCCCTCGCCTGCGACATCCGCATCGCCGAGCGCCAGGCGCAGATGGCCCTGCCGGAGGCCGCGGTGGGCCTGCTGCCCTGCGCCGGCGGGACCCAGGCGCTGCCCTGGCTGGTGGGCGAAGGCTGGGCCAAGCGGATGATCCTCTGCAACGAGCGGGTGGATGCGGAAACCGCCCTGCGCATCGGCCTGGTCGAACAGGTGGTGGACAGCGGCGAGGCGCGCGGCGCCGCCCTGCTGCTGGCGGCCAAGGTGGCACGCCAGAGCCCGGTGGCGATCCGCACCATCAAGCCGCTGATCCAGGGTGCCCGCGAACGCGCGCCGAACACTTGGCTGCCGGAGGAGCGCGAGCGCTTCGTCGATCTGTTCGACGCCCAGGACACCCGCGAAGGGGTCAACGCCTTCCTCGAGAAGCGCGATCCCAAGTGGCGCAACTGCTGAGCCTTCCCATCGAACCCGGCGGGCTGCGCCACAGCCCGCCCGAGCCGAGAGCCCCGACATGAACGTGCTTTTCGAAGAACGCCCGAGCCTGCATGGCTTCCGCATCGGCATTGCCACCCTGGATGCGGAAAAGAGCCTGAACGCCCTGTCCCTGCCGATGATCGAGGCCCTGGCGGCGAAACTCGACGCCTGGGCCGAGGACGCCGGCATCGCCTGCGTGCTGTTGCGCGGCAACGGCGCCAAGGCCTTTTGCGCCGGCGGCGACGTGCGCAAGCTGGTGGACGCCTGCCGTGAGCAACCCGGCGAGGTACCGGCCCTGGCGCGACGCTTCTTCGCCGATGAATACCGCCTCGACTACCGCATCCACACCTACCCGAAACCCTTCATCTGCTGGGCCCACGGCTATGTGATGGGCGGCGGCATGGGGCTGATGCAGGGTGCCGGGATTCGCATCGTCACCCCGTCCAGCCGCCTGGCGATGCCGGAGATCGGCATCGGCCTGTACCCAGACGTCGGCGCCAGCTGGTTCCTCGCCCGCCTGCCCGGCAGGCTCGGCCTGTTCCTCGGCCTGAGCGCGGCGCAGATGAACGCCCGCGACGCTCTCGACCTCGACCTGGCCGACCGCTTCCTTCTCGACGACCAGCAGGACGCCCTGCTCGCCGGCCTGGTGCAGATGAACTGGAACGAGTCGCCCCAGGTGCAACTTCACAGCCTGCTTCGCGCCCTCGAACACGAGGCCCGCGGCGAACTGCCGGAGGCCCAGTTGCTCCCGCGCCGGCCGCGCCTGGACGCACTGCTCGACCAGCCGGACCTGGCGAGCGCCTGGCAGGCGCTGGTGGCGCTGCGCGACGACGCCGACCCGCTGCTTGCGCGTGGCGCGAAGACCCTGGCCGAGGGCTGCCCGATGACCGCGCATCTGGTCTGGCAGCAGATCGAGCGTGCGCGCTACCTGTCGCTGGCCGAGGTGTTCCGCCTGGAATACGCCATGAGCCTGAACTGCACGCGCCATCCGGACTTCGCCGAGGGGGTACGCGCAAGGCTGATCGATCGCGACAACGCACCGAACTGGCATTGGCCGCAGGTGGAATCGATCCCGCAGGCGGTGATCGAGGCGCACTTCGAGCCGACCTGGGAGGGCGAGCATCCGCTGGCGGGGCTCTGAGCTAGCAAGGAGATATACCATGCAAGACCTCGAACTGAGCGAAGAACAGATCATGATCCGCGACATGGCCCGGGACTTTGCCCGCGGCGAAATCGCACCCCATGCCCAGGCCTGGGAAAAGGCCGGCTGGATCGATGATGGCGTGGTGCGCAAGATGGGCGAACTGGGCCTGTTGGGCATGGTGGTGCCCGAGGACTTCGGCGGCAGCTACACCGACTATGTCGCCTATGCCCTGGCCGTGGAAGAGATCTCCGCCGGCTGCGGCGCCACCGGGGCGATGATGAGCATCCACAACTCGGTAGGTTGCGGCCCGCTGCTGGCCTACGGTACCGCCGAACAGCAACAGCAGTGGCTGCCTCGCCTGGCCAGCGGCGAGGTGATTGGCTGCTTCTGCCTGACCGAGCCCCAGGCCGGCTCCGAGGCGCACAACCTGCGCACCCGCGCCGAGCTGGTGGATGGCCAGTGGGTCATCAACGGTGCCAAGCAGTTCGTCAGCAACGCCCGCCGTGCCGGCCTGGCGATCGTCTTCGCAGTGACCGACCCGGAGCTGGGCAAGAAAGGCCTGTCGGCGTTCCTGGTGCCCACCGACAACCCTGGGTTCAAGGTTGACCGCAGCGAACACAAGATGGGCATTCGCGCCTCCGATACCTGCGCGGTCACCTTCGACAACTGCCGCATTCCTGCCGCCAATATCCTCGGTGAGCGTGGCAAGGGCCTGGCCATTGCCCTGTCCAACCTCGAAGGCGGCCGTATCGGCATTGCCGCCCAGGCCTTGGGCATCGCCCGCGCCGCATTCGAAGCGGCGCTGGTCTACTCGCGTGACCGCATACAGTTTGGCAAGCCGATCAACGAACACCAGAGCATTGCCAACCTGCTGGCCGACATGCAGGTGCAGGTGAACGCCGCACGTTTGCTGATCCTGCATGCCGCCCGCCTGCGCAGTGCCGGCAAGCCGTGCCTGTCGGAAGCGTCACAGGCCAAGCTGTTCGCTTCGGAAATGGCCGAGCGGGTGTGCTCGATGGCAATCCAGGTGCATGGTGGGTACGGTTATCTGGAAGATTACCCGGTGGAGCGCTATTACCGGGATGCGCGGATTACCCAGATCTATGAAGGGTCGAGCGAGATCCAGCGGATGCTGATTGCGCGGGAATTGAAAAACTATCCGCTGTAACATATGACTAGTTATAATTTTGCGGCCGCTTAAGATGGAGATATACCATGAGTGATTACGAGCCGTTGCGTCTGCATGTCCCGGAGCCCACCGGGCGTCCTGGCTGCAAGACCGACTTTTCCTATCTGCACCTGTCCCCCGCCGGCGAGGTACGCAAGCCGCCGGTGGATGTCGAGCCCGCCGAGACCAGCGACCTGGCCTACAGCCTGGTACGTGTGCTCGACGACGACGGCCACGCCGTCGGTCCCTGGAATCCGCAGCTCAGCAACGAACAACTGCTGCGCGGCATGCGGGCGATGCTCAAGACCCGCCTGTTCGACGCGCGCATGCTCACCGCGCAACGGCAGAAAAAGCTTTCCTTCTATATGCAATGCCTCGGCGAGGAAGCCATCGCCACCGCCCACACCCTGGCCCTGCGCGACGGCGACATGTGCTTTCCGACCTATCGCCAGCAAGGCATCCTGATCACCCGCGAATACCCGCTGGTGGACATGATCTGCCAGCTTCTCTCCAACGAGGCCGACCCGCTCAAGGGCCGCCAGCTGCCGATCATGTACTCGAGCAAGGAGGCAGGTTTCTTCTCCATCTCCGGCAACCTCGCCACCCAGTTCATCCAGGCGGTCGGCTGGGGCATGGCCTCGGCGATCAAGGGCGACACGCGCATCGCCTCGGCCTGGATCGGCGACGGCGCCACCGCCGAGTCGGACTTCCACACCGCCCTCACCTTCGCCCATGTCTACCGCGCGCCGGTAATCCTCAACGTGGTCAACAACCAGTGGGCGATCTCCACCTTCCAGGCCATCGCCGGCGGCGAAGGCACCACCTTCGCCAACCGTGGCGTGGGCTGCGGGATCGCCTCGCTGCGGGTCGACGGCAATGACTTCCTGGCGGTCTACGCCGCCTCCGAGTGGGCCGCCGAGCGCGCCCGGCGCAACCTCGGGCCGAGCCTGATCGAATGGGTCACCTACCGCGCCGGCCCGCACTCGACTTCGGACGACCCGTCCAAGTACCGCCCCGCCGACGACTGGACCAACTTCCCGCTGGGCGACCCGATCGCCCGCCTGAAGCGGCACATGATCGGCCTCGGCATCTGGTCGGAGGAACAGCACGAAGCCACCCACAAGGCCCTCGAAGCCGAAGTACTGGCCGCGCAGAAACAGGCGGAGAGCCATGGCACCCTGATCGACGGCCGGGTGCCGAGCGCCGCCAGCATGTTCGAGGACGTCTATGCAGAACTGCCGGAGCACCTGCGCCGGCAACGCCAGGAGCTCGGGGTATGAATGCCATGAACCCGCAACACGAGAACGCCCAGACGGTCACCAGCATGACCATGATCCAGGCGCTGCGCTCGGCGATGGACATCATGCTCGAGCGCGACGACGACGTGGTGGTATTCGGCCAGGACGTCGGCTACTTCGGCGGCGTGTTCCGCTGCACCGAAGGCCTGCAGAAGAAATACGGCACCTCGCGGGTGTTCGATGCGCCGATCTCCGAGAGCGGCATCATCGGCGCCGCGGTCGGCATGGGTGCCTACGGCCTGCGCCCGGTGGTGGAGATCCAGTTCGCCGACTACGTCTACCCGGCCTCCGACCAGTTGATCTCCGAGGCGGCGCGCCTGCGCTATCGCTCGGCCGGCGACTTCATCGTGCCGATGACCGTACGCATGCCCTGTGGCGGCGGCATCTACGGCGGGCAAACGCACAGCCAGAGCCCGGAGGCGATGTTCACCCAGGTCTGCGGCCTGCGCACGGTGATGCCGTCCAACCCCTACGACGCCAAGGGCCTGCTGATCGCCTGCATCGAGAACGACGACCCGGTGATCTTCCTCGAGCCCAAGCGCCTCTACAACGGCCCGTTCGATGGCCACCACGACCGCCCGGTGACGCCCTGGTCCAAGCATCCGGCCAGCCAGGTGCCGGACGGCTACTACAAGGTGCCGCTGGACAAGGCGGCGATCGTCCGCCCCGGCGCGGCGCTGACCGTGCTGACCTACGGCACCATGGTCTACGTGGCCCAGGCCGCGGCCGACGAGACCGGCCTGGACGCCGAGATCATCGACCTGCGCAGCCTCTGGCCGCTGGACCTGGAAACCATCGTCGCCTCGGTGAAGAAGACCGGCCGCTGCGTCATCGCCCACGAGGCGACCCGCACCTGCGGGTTCGGCGCCGAGCTGATGTCGCTGGTGCAGGAGCACTGCTTCCACCACCTGGAGGCGCCGATCGAGCGCGTCACCGGTTGGGACACCCCCTACCCGCATGCCCAGGAGTGGGCGTATTTCCCCGGCCCCGCGCGCGTCGGCGCGGCATTCAAGCGTGTGATGGAGGTCTGAATGGGTACCCATGTGATCAAGATGCCGGACATCGGGGAAGGCATCGCCGAGGTCGAACTGGTGGAGTGGCATGTCCAGGTCGGCGACTCGGTCAATGAAGACCAGGTCCTCGCCGAGGTGATGACCGACAAGGCCACGGTGGAGATTCCCTCGCCGGTGGCCGGACGCATCCTCGCCCTCGGCGGCCAGCCGGGCCAGGTGATGGCGGTGGGCGGCGAACTGATCCGCCTGGAGGTGGAAGGCGCCGGCAACCTCGCCGAGAGTCCGGCCGCGGCGACGCCGGCCGCGCCCGTCGCCGCCACCCCGGAGAAACCGAAGGAAGCCCCGGTCGCGGCGCCGAAAGCCGCCGCCGAAGCGCCGCGCGCCTTGCGCGACAGCGAGGCGCCACGGCAGCGGCGCCAGCCCGGCGAACGCCCGCTGGCCTCCCCCGCGGTGCGCCAGCGCGCCCGCGACCTGGGCATCGAGTTGCAGTTCGTGCAGGGCAGCGGTCCCGCCGGACGCGTCCTCCACGAGGACCTCGATGCCTACCTGACCCAGGATGGCAGCGTCGCGCGCAGCGGCGGCGCCGCGCAGGGGTATGCCGAGCGACACGACGAACAGGCGGTGCCGGTGATCGGCCTGCGTCGCAAGATCGCCCAGAAGATGCAGGACGCCAAGCGACGCATCCCGCATTTCAGCTATGTCGAGGAAATCGACGTCACCGATCTGGAAGCCCTGCGCGCCCATCTCAACCAGAAATGGGGTGGCCAGCGCGGCAAGCTGACCCTGCTGCCGTTCCTGGTCCGCGCCATGGTCGTGGCGCTGCGCGACTTCCCGCAGTTGAACGCGCGCTACGACGACGAGGCCGAGGTGGTCACCCGCTACGGCGCGGTGCACGTCGGCATCGCCACCCAGAGCGACAACGGCCTGATGGTGCCGGTGCTGCGCCACGCCGAATCGCGCGACCTCTGGGGCAACGCCAGCGAAGTGGCGCGCCTGGCCGAAGCCGCACGCAGCGGCAAGGCGCAACGCCAGGAGCTGTCCGGCTCGACCATCACCCTGAGCAGCCTCGGCGTGCTCGGCGGGATCGTCAGCACACCGGTGATCAACCATCCGGAGGTGGCCATCGTCGGCGTCAACCGCATCGTCGAGCGACCGATGGTGGTCGGCGGCAACATCGTCGTGCGCAAGATGATGAACCTCTCCTCCTCCTTCGACCACCGGGTGGTCGACGGGATGGACGCGGCGGCCTTCATCCAGGCCGTGCGCGGCCTGCTCGAACATCCCGCCACCCTGTTCCTGGAGTAAGCGATGAGCCAGATCCTGAAGACTTCCCTGCTGATCGTCGGCGGCGGTCCCGGCGGCTACGTCGCGGCGATCCGTGCCGGGCAACTGGGCATTCCCACCGTACTGGTGGAGGGCGCCGCCCTCGGCGGCACCTGCCTGAACGTCGGCTGCATCCCGTCGAAGGCGCTGATCCACGCCGCCGAGGAATACCTCAAGGCCCGCCACTATGCCAGCCGGTCGGCGCTGGGCATCCAGGTACAGGCGCCGAGCATCGACATCGCCCGCACCGTGGAATGGAAGGACGCCATCGTCGACCGCCTCACCAGCGGCGTCGCCGCGCTGCTGAAGAAACACGGGGTCGATGTCGTCCAGGGCTGGGCGAGGATCCTCGACGGCAAAAGCGTGGCGGTCGAACTCGCCGGCGGCGGCAGCCAGCGCATCGAGTGCGAGCACCTGCTGCTGGCCGCCGGCTCGCAGAGCGTCGAGCTACCGATCCTGCCGCTGGGCGGCAAGGTGATCTCCTCCACCGAGGCGCTGGCGCCCGGCAGCCTGCCCAAGCGCCTGGTGGTGGTCGGCGGCGGCTACATCGGCCTGGAGCTGGGTACCGCCTACCGCAAGCTCGGCGTCGAGGTGGCGGTGGTGGAAGCGCAACCACGCATCCTGCCGGGCTACGACGAAGAACTGACCAAGCCGGTGGCCCAGGCCTTGCGCAGGCTGGGCGTCGAGCTGTACCTCGGGCACAGCCTGCTGGGCCCGAGCGAGAACGGCGTGCGGGTCCGCGACGGCGCCGGCGAGGAGCGCGAGATCGCCGCCGACCAGGTACTGGTGGCGGTCGGCCGCAAGCCGCGCAGCGAAGGCTGGAACCTGGAAAGCCTGGGCCTGGACATGAACGGCCGGGCGGTGAAGGTCGACGACCAGTGCCGCACCTCGATGCGCAATGTCTGGGCCATAGGCGATCTCGCCGGCGAGCCGATGCTCGCGCACCGGGCCATGGCCCAGGGCGAGATGGTCGCCGAGCTGATCGCCGGCAAGCGTCGCCAGTTCGCCCCGGTGGCGATCCCCGCGGTGTGCTTCACCGATCCGGAAGTGGTGGTCGCCGGGTTGTCCCCGGAGCAGGCGAAGGATGCCGGCCTGGACTGCCTGGTGGCGAGCTTCCCGTTCGCCGCCAACGGTCGCGCCATGACCCTGGAGGCCAACGAAGGCTTCGTCCGCGTGGTGGCGCGTCGCGACAACCACCTGGTCGTCGGCTGGCAGGCGGTGGGCAAGGCGGTCTCGGAACTGTCCACGGCCTTCGCCCAGTCGCTGGAGATGGGCGCCCGCCTGGAAGACATCGCCGGCACCATCCACGCCCATCCGACCCTCGGCGAAGCGGTCCAGGAAGCCGCCCTGCGCGCGCTGGGACACGCCCTGCACATCTGATCTAGAGTCGACCTGCAGGCATGCAAGCTTGGCTGTTTTGGCGGATGAGAGAAGATTTTCAGCCTGATACAGATTAAATCAGAACGCAGAAGCGGTCTGATAAAACAGAATTTGCCTGGCGGCAGTAGCGCGGTGGTCCCACCTGACCCCATGCCGAACTCAGAAGTGAAACGCCGTAGCGCCGATGGTAGTGTGGGGTCTCCCCATGCGAGAGTAGGGAACTGCCAGGCATCAAATAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTATCTGTTGTTTGTCGGTGAACGCTCTCCTGAGTAGGACAAATCCGCCGGGAGCGGATTTGAACGTTGCGAAGCAACGGCCCGGAGGGTGGCGGGCAGGACGCCCGCCATAAACTGCCAGGCATCAAATTAAGCAGAAGGCCATCCTGACGGATGGCCTTTTTGCGTTTCTACAAACTCTTTTGTTTATTTTTCTAAATACATTCAAATATGTATCCGCTCATGAGACAATAACCCTGATAAATGCTTCAATAATATTGAAAAAGGAAGAGTATGAGTATTCAACATTTCCGTGTCGCCCTTATTCCCTTTTTTGCGGCATTTTGCCTTCCTGTTTTTGCTCACCCAGAAACGCTGGTGAAAGTAAAAGATGCTGAAGATCAGTTGGGTGCACGAGTGGGTTACATCGAACTGGATCTCAACAGCGGTAAGATCCTTGAGAGTTTTCGCCCCGAAGAACGTTTTCCAATGATGAGCACTTTTAAAGTTCTGCTATGTGGCGCGGTATTATCCCGTGTTGACGCCGGGCAAGAGCAACTCGGTCGCCGCATACACTATTCTCAGAATGACTTGGTTGAGTACTCACCAGTCACAGAAAAGCATCTTACGGATGGCATGACAGTAAGAGAATTATGCAGTGCTGCCATAACCATGAGTGATAACACTGCGGCCAACTTACTTCTGACAACGATCGGAGGACCGAAGGAGCTAACCGCTTTTTTGCACAACATGGGGGATCATGTAACTCGCCTTGATCGTTGGGAACCGGAGCTGAATGAAGCCATACCAAACGACGAGCGTGACACCACGATGCCTGTAGCAATGGCAACAACGTTGCGCAAACTATTAACTGGCGAACTACTTACTCTAGCTTCCCGGCAACAATTAATAGACTGGATGGAGGCGGATAAAGTTGCAGGACCACTTCTGCGCTCGGCCCTTCCGGCTGGCTGGTTTATTGCTGATAAATCTGGAGCCGGTGAGCGTGGGTCTCGCGGTATCATTGCAGCACTGGGGCCAGATGGTAAGCCCTCCCGTATCGTAGTTATCTACACGACGGGGAGTCAGGCAACTATGGATGAACGAAATAGACAGATCGCTGAGATAGGTGCCTCACTGATTAAGCATTGGTAACTGTCAGACCAAGTTTACTCATATATACTTTAGATTGATTTACGCGCCCTGTAGCGGCGCATTAAGCGCGGCGGGTGTGGTGGTTACGCGCAGCGTGACCGCTACACTTGCCAGCGCCCTAGCGCCCGCTCCTTTCGCTTTCTTCCCTTCCTTTCTCGCCACGTTCGCCGGCTTTCCCCGTCAAGCTCTAAATCGGGGGCTCCCTTTAGGGTTCCGATTTAGTGCTTTACGGCACCTCGACCCCAAAAAACTTGATTTGGGTGATGGTTCACGTAGTGGGCCATCGCCCTGATAGACGGTTTTTCGCCCTTTGACGTTGGAGTCCACGTTCTTTAATAGTGGACTCTTGTTCCAAACTTGAACAACACTCAACCCTATCTCGGGCTATTCTTTTGATTTATAAGGGATTTTGCCGATTTCGGCCTATTGGTTAAAAAATGAGCTGATTTAACAAAAATTTAACGCGAATTTTAACAAAATATTAACGTTTACAATTTAAAAGGATCTAGGTGAAGATCCTTTTTGATAATCTCATGACCAAAATCCCTTAACGTGAGTTTTCGTTCCACTGAGCGTCAGACCCCGTAGAAAAGATCAAAGGATCTTCTTGAGATCCTTTTTTTCTGCGCGTAATCTGCTGCTTGCAAACAAAAAAACCACCGCTACCAGCGGTGGTTTGTTTGCCGGATCAAGAGCTACCAACTCTTTTTCCGAAGGTAACTGGCTTCAGCAGAGCGCAGATACCAAATACTGTCCTTCTAGTGTAGCCGTAGTTAGGCCACCACTTCAAGAACTCTGTAGCACCGCCTACATACCTCGCTCTGCTAATCCTGTTACCAGTGGCTGCTGCCAGTGGCGATAAGTCGTGTCTTACCGGGTTGGACTCAAGACGATAGTTACCGGATAAGGCGCAGCGGTCGGGCTGAACGGGGGGTTCGTGCACACAGCCCAGCTTGGAGCGAACGACCTACACCGAACTGAGATACCTACAGCGTGAGCTATGAGAAAGCGCCACGCTTCCCGAAGGGAGAAAGGCGGACAGGTATCCGGTAAGCGGCAGGGTCGGAACAGGAGAGCGCACGAGGGAGCTTCCAGGGGGAAACGCCTGGTATCTTTATAGTCCTGTCGGGTTTCGCCACCTCTGACTTGAGCGTCGATTTTTGTGATGCTCGTCAGGGGGGCGGAGCCTATGGAAAAACGCCAGCAACGCGGCCTTTTTACGGTTCCTGGCCTTTTGCTGGCCTTTTGCTCACATGTTCTTTCCTGCGTTATCCCCTGATTCTGTGGATAACCGTATTACCGCCTTTGAGTGAGCTGATACCGCTCGCCGCAGCCGAACGACCGAGCGCAGCGAGTCAGTGAGCGAGGAAGCGGAAGAGCGCCTGATGCGGTATTTTCTCCTTACGCATCTGTGCGGTATTTCACACCGCATAGGGTCATGGCTGCGCCCCGACACCCGCCAACACCCGCTGACGCGCCCTGACGGGCTTGTCTGCTCCCGGCATCCGCTTACAGACAAGCTGTGACCGTCTCCGGGAGCTGCATGTGTCAGAGGTTTTCACCGTCATCACCGAAACGCGCGAGGCAGCAAGGAGATGGCGCCCAACAGTCCCCCGGCCACGGGGCCTGCCACCATACCCACGCCGAAACAAGCGCTCATGAGCCCGAAGTGGCGAGCCCGATCTTCCCCATCGGTGATGTCGGCGATATAGGCGCCAGCAACCGCACCTGTGGCGCCGGTGATGCCGGCCACGATGCGTCCGGCGTAGAGGATCTGCTCATGTTTGACAGCTTATC'

In [21]:
source.find(gene.upper())

8148

#### Reading template plasmids

In [2]:
#import fasta files
fastas = ['promoters', 'gates']
plasmids = pd.DataFrame([(p.id, str(p.seq)) for fasta in fastas \
                        for p in list(SeqIO.parse('datasets/jump/{}.fasta'.format(fasta), 'fasta'))], \
                        columns=['name', 'sequence'])
plasmids['sequence'] = plasmids['sequence'].str.upper()
plasmids.head()

,name,sequence
0,3K3-araC-PBAD-30-gfp-B15,TGCCACCTGACGTCTAAGAAACCATTATTATCATGACATTAACCTA...
1,3K3-araC-PBAD-(rJ)30-gfp-B15,TGCCACCTGACGTCTAAGAAACCATTATTATCATGACATTAACCTA...
2,3K3-cymR-PcymRC-(rJ)30-gfp-B15,TGCCACCTGACGTCTAAGAAACCATTATTATCATGACATTAACCTA...
3,3K3-PBAD-(rJ)30-gfp-B15,TGCCACCTGACGTCTAAGAAACCATTATTATCATGACATTAACCTA...
4,3K3-Pcin-(rJ)30-gfp-B15,TGCCACCTGACGTCTAAGAAACCATTATTATCATGACATTAACCTA...


In [3]:
#add plasmid-ids
plasmids = pd.merge(pd.read_csv('datasets/dictionary.csv'), plasmids, \
                    left_on='full_name', right_on='name', how='right')[['id', 'short_name', 'name', 'sequence']]
plasmids.head()

,id,short_name,name,sequence
0,P73,araC-PBAD-noRJ,3K3-araC-PBAD-30-gfp-B15,TGCCACCTGACGTCTAAGAAACCATTATTATCATGACATTAACCTA...
1,P63,araC-PBAD,3K3-araC-PBAD-(rJ)30-gfp-B15,TGCCACCTGACGTCTAAGAAACCATTATTATCATGACATTAACCTA...
2,P66,cymR-PCymRC,3K3-cymR-PcymRC-(rJ)30-gfp-B15,TGCCACCTGACGTCTAAGAAACCATTATTATCATGACATTAACCTA...
3,P62,PBAD,3K3-PBAD-(rJ)30-gfp-B15,TGCCACCTGACGTCTAAGAAACCATTATTATCATGACATTAACCTA...
4,P64,Pcin,3K3-Pcin-(rJ)30-gfp-B15,TGCCACCTGACGTCTAAGAAACCATTATTATCATGACATTAACCTA...


In [4]:
#import parts
parts = pd.read_csv('datasets/jump/parts.csv')
parts['sequence'] = parts['sequence'].str.upper()
parts.head()

,name,overhang,sequence,color
0,P11,P,GCCTCCACACCGCTCGTCACATCCTGTGATCCATTCCCCGCCCATC...,#f26520
1,E11-N,NOC,ATGAGCGATAGTCCGCAGAAACTGGGTCGTAATGAATGGAATGCAT...,#f26520
2,E11-C,NOC,ACAGATGTAACGATTAAAAGAATAATATCCAAAGGAGAACTTGAAT...,#f26520
3,P20,P,GCGCGGATAAAAATTTCATTTGCCCGCGACGGTTTTTCCGCCCATC...,#e23322
4,E20-N,NOC,AATGAAACCGATCCTGATCTGGAACTGCTGAAACGTATTGGTAATA...,#e23322


In [17]:
dictionary = pd.read_csv('datasets/dictionary.csv')
mapping = dict(zip(dictionary['full_name'], dictionary['short_name']))

In [20]:
templates = pd.DataFrame([(part['name'], plasmid['name']) for _, part in parts.iterrows() \
                      for _, plasmid in plasmids.iterrows() if plasmid['sequence'].find(part['sequence'])!=-1], \
                      columns=['name', 'template'])
templates['template'] = templates['template'].map(mapping)
templates = templates.groupby('name')['template'] \
                      .apply(lambda x: ', '.join(x)).reset_index()
templates

,name,template
0,E11-C,e11x32STPhoRadA
1,E11-N,e11x32STPhoRadA
2,E20-C,e20x32gp411
3,E20-N,e20x32gp411
4,E38-C,e38x32gp418
5,E38-N,e38x32gp418
6,E42-C,e42x32STIMPDH1
7,E42-N,e42x32STIMPDH1
8,GFPmut3b,"araC-PBAD-noRJ, araC-PBAD, cymR-PCymRC, PBAD, ..."
9,P11,e11x32STPhoRadA


In [5]:
pd.DataFrame([(part['name'], plasmid['name']) for _, part in parts.iterrows() \
                      for _, plasmid in plasmids.iterrows() if plasmid['sequence'].find(part['sequence'])!=-1], \
                      columns=['name', 'template'])

,name,template
0,P11,3K3-P11*-gfp-B15-P(ara)-sECF11*-SP11-P(Cym)-32...
1,E11-N,3K3-P11*-gfp-B15-P(ara)-sECF11*-SP11-P(Cym)-32...
2,E11-C,3K3-P11*-gfp-B15-P(ara)-sECF11*-SP11-P(Cym)-32...
3,P20,3K3-P20*-gfp-B15-P(ara)-sECF20*-SP3-P(Cym)-32-...
4,E20-N,3K3-P20*-gfp-B15-P(ara)-sECF20*-SP3-P(Cym)-32-...
...,...,...
75,PCymRC,3K3-P41*-gfp-B15-P(ara)-sECF41*-SP5-P(Cym)-32-...
76,PCymRC,3K3-P42*-gfp-B15-P(ara)-sECF42*-SP6-P(Cym)-32-...
77,PSalTTC,3K3-PsalTTC-(rJ)30-gfp-B15
78,PCin,3K3-Pcin-(rJ)30-gfp-B15


In [6]:
template = pd.DataFrame([(part['name'], plasmid['name']) for _, part in parts.iterrows() \
                      for _, plasmid in plasmids.iterrows() if plasmid['sequence'].find(part['sequence'])!=-1], \
                      columns=['name', 'template']).groupby('name')['template'] \
                      .apply(lambda x: ', '.join(x)).reset_index()
template

,name,template
0,E11-C,3K3-P11*-gfp-B15-P(ara)-sECF11*-SP11-P(Cym)-32...
1,E11-N,3K3-P11*-gfp-B15-P(ara)-sECF11*-SP11-P(Cym)-32...
2,E20-C,3K3-P20*-gfp-B15-P(ara)-sECF20*-SP3-P(Cym)-32-...
3,E20-N,3K3-P20*-gfp-B15-P(ara)-sECF20*-SP3-P(Cym)-32-...
4,E38-C,3K3-P38*-gfp-B15-P(ara)-sECF38*-SP4-P(Cym)-32-...
5,E38-N,3K3-P38*-gfp-B15-P(ara)-sECF38*-SP4-P(Cym)-32-...
6,E42-C,3K3-P42*-gfp-B15-P(ara)-sECF42*-SP6-P(Cym)-32-...
7,E42-N,3K3-P42*-gfp-B15-P(ara)-sECF42*-SP6-P(Cym)-32-...
8,GFPmut3b,"3K3-araC-PBAD-30-gfp-B15, 3K3-araC-PBAD-(rJ)30..."
9,P11,3K3-P11*-gfp-B15-P(ara)-sECF11*-SP11-P(Cym)-32...


List of parts that are not found in template plasmids

In [7]:
missing = list(set(parts['name'].tolist()).difference(set(template['name'].tolist())))
missing

['J23101']

#### Parts Domestication

In [13]:
forbidden_sites = np.array([(enz, str(Seq(enz).reverse_complement())) \
                            for enz in [BsaI.site, BsmBI.site]]).ravel().tolist()
existing_parts = parts[~parts['name'].isin(missing)]

#check any forbidden sites
existing_parts['num_forbidden_sites'] = existing_parts['sequence'].apply(lambda x: np.sum([x.count(a) for a in forbidden_sites]))
existing_parts.reset_index(drop=True)

,name,overhang,sequence,color,num_forbidden_sites
0,P11,P,GCCTCCACACCGCTCGTCACATCCTGTGATCCATTCCCCGCCCATC...,#f26520,0
1,E11-N,NOC,ATGAGCGATAGTCCGCAGAAACTGGGTCGTAATGAATGGAATGCAT...,#f26520,0
2,E11-C,NOC,ACAGATGTAACGATTAAAAGAATAATATCCAAAGGAGAACTTGAAT...,#f26520,0
3,P20,P,GCGCGGATAAAAATTTCATTTGCCCGCGACGGTTTTTCCGCCCATC...,#e23322,0
4,E20-N,NOC,AATGAAACCGATCCTGATCTGGAACTGCTGAAACGTATTGGTAATA...,#e23322,0
5,E20-C,NOC,CTGAAAAAAATCCTGAAAATCGAGGAACTGGATGAACGCGAACTGA...,#e23322,0
6,P38,P,CAGTACAAAATTTTTTAGATGCGTTGTACAACCCTCACGGGGGTGG...,#81171b,0
7,E38-N,NOC,CCTGCCGCACGTCCGGCACGTATTACCAATCAGCGTGATGGTGCAG...,#81171b,0
8,E38-C,NOC,TGTGAAATCTTTGAAAACGAGATCGACTGGGATGAAATTGCCAGCA...,#81171b,0
9,P42,P,CAGTACAAAATTTTTTAGATGCGTTCGCTGTCGATCCGGCCCGTCG...,#ca4642,0


In [11]:
parts['num_forbidden_sites'] = parts['sequence'].apply(lambda x: np.sum([x.count(a) for a in forbidden_sites]))
parts

,name,overhang,sequence,color,num_forbidden_sites
0,P11,P,GCCTCCACACCGCTCGTCACATCCTGTGATCCATTCCCCGCCCATC...,#f26520,0
1,E11-N,NOC,ATGAGCGATAGTCCGCAGAAACTGGGTCGTAATGAATGGAATGCAT...,#f26520,0
2,E11-C,NOC,ACAGATGTAACGATTAAAAGAATAATATCCAAAGGAGAACTTGAAT...,#f26520,0
3,P20,P,GCGCGGATAAAAATTTCATTTGCCCGCGACGGTTTTTCCGCCCATC...,#e23322,0
4,E20-N,NOC,AATGAAACCGATCCTGATCTGGAACTGCTGAAACGTATTGGTAATA...,#e23322,0
5,E20-C,NOC,CTGAAAAAAATCCTGAAAATCGAGGAACTGGATGAACGCGAACTGA...,#e23322,0
6,P38,P,CAGTACAAAATTTTTTAGATGCGTTGTACAACCCTCACGGGGGTGG...,#81171b,0
7,E38-N,NOC,CCTGCCGCACGTCCGGCACGTATTACCAATCAGCGTGATGGTGCAG...,#81171b,0
8,E38-C,NOC,TGTGAAATCTTTGAAAACGAGATCGACTGGGATGAAATTGCCAGCA...,#81171b,0
9,P42,P,CAGTACAAAATTTTTTAGATGCGTTCGCTGTCGATCCGGCCCGTCG...,#ca4642,0


In [14]:
#find parts with any forbidden sites
invalid_parts = parts[parts['num_forbidden_sites']>0]
invalid_parts['name'].tolist()

['GFPmut3b']

In [15]:
removed = invalid_parts['name'].tolist() + missing
valid_parts = parts[~parts['name'].isin(removed)].reset_index(drop=True)
valid_parts.head()

,name,overhang,sequence,color,num_forbidden_sites
0,P11,P,GCCTCCACACCGCTCGTCACATCCTGTGATCCATTCCCCGCCCATC...,#f26520,0
1,E11-N,NOC,ATGAGCGATAGTCCGCAGAAACTGGGTCGTAATGAATGGAATGCAT...,#f26520,0
2,E11-C,NOC,ACAGATGTAACGATTAAAAGAATAATATCCAAAGGAGAACTTGAAT...,#f26520,0
3,P20,P,GCGCGGATAAAAATTTCATTTGCCCGCGACGGTTTTTCCGCCCATC...,#e23322,0
4,E20-N,NOC,AATGAAACCGATCCTGATCTGGAACTGCTGAAACGTATTGGTAATA...,#e23322,0


In [68]:
valid_parts.shape

(18, 5)

#### Design Overhangs & Primers

In [92]:
fragments = pd.merge(valid_parts, pd.read_csv('datasets/jump/overhang.csv'), on='overhang', how='left') \
                [['name', 'sequence', 'left_site', 'right_site']]
prefix, suffix = forbidden_sites[2] + forbidden_sites[0] + 'A', 'T' + forbidden_sites[1] + 'T' + forbidden_sites[3]
fragments['left_overhang'] = prefix + fragments['left_site']
fragments['right_overhang'] = fragments['right_site'] + suffix
fragments['ext_sequence'] = fragments['left_overhang'] + fragments['sequence'] + fragments['right_overhang']
#following benchling convention to start from index 1
fragments['size'] = fragments['ext_sequence'].apply(lambda x: len(x)+1)
#if need to print out the result for benchling import
#fragments.to_csv('datasets/jump/fragment-output.csv')
fragments

,name,sequence,left_site,right_site,left_overhang,right_overhang,ext_sequence,size
0,P11,GCCTCCACACCGCTCGTCACATCCTGTGATCCATTCCCCGCCCATC...,GGAG,TACT,CGTCTCGGTCTCAGGAG,TACTTGAGACCTGAGACG,CGTCTCGGTCTCAGGAGGCCTCCACACCGCTCGTCACATCCTGTGA...,97
1,E11-N,ATGAGCGATAGTCCGCAGAAACTGGGTCGTAATGAATGGAATGCAT...,AATG,GCTT,CGTCTCGGTCTCAAATG,GCTTTGAGACCTGAGACG,CGTCTCGGTCTCAAATGATGAGCGATAGTCCGCAGAAACTGGGTCG...,768
2,E11-C,ACAGATGTAACGATTAAAAGAATAATATCCAAAGGAGAACTTGAAT...,AATG,GCTT,CGTCTCGGTCTCAAATG,GCTTTGAGACCTGAGACG,CGTCTCGGTCTCAAATGACAGATGTAACGATTAAAAGAATAATATC...,468
3,P20,GCGCGGATAAAAATTTCATTTGCCCGCGACGGTTTTTCCGCCCATC...,GGAG,TACT,CGTCTCGGTCTCAGGAG,TACTTGAGACCTGAGACG,CGTCTCGGTCTCAGGAGGCGCGGATAAAAATTTCATTTGCCCGCGA...,96
4,E20-N,AATGAAACCGATCCTGATCTGGAACTGCTGAAACGTATTGGTAATA...,AATG,GCTT,CGTCTCGGTCTCAAATG,GCTTTGAGACCTGAGACG,CGTCTCGGTCTCAAATGAATGAAACCGATCCTGATCTGGAACTGCT...,624
5,E20-C,CTGAAAAAAATCCTGAAAATCGAGGAACTGGATGAACGCGAACTGA...,AATG,GCTT,CGTCTCGGTCTCAAATG,GCTTTGAGACCTGAGACG,CGTCTCGGTCTCAAATGCTGAAAAAAATCCTGAAAATCGAGGAACT...,444
6,P38,CAGTACAAAATTTTTTAGATGCGTTGTACAACCCTCACGGGGGTGG...,GGAG,TACT,CGTCTCGGTCTCAGGAG,TACTTGAGACCTGAGACG,CGTCTCGGTCTCAGGAGCAGTACAAAATTTTTTAGATGCGTTGTAC...,116
7,E38-N,CCTGCCGCACGTCCGGCACGTATTACCAATCAGCGTGATGGTGCAG...,AATG,GCTT,CGTCTCGGTCTCAAATG,GCTTTGAGACCTGAGACG,CGTCTCGGTCTCAAATGCCTGCCGCACGTCCGGCACGTATTACCAA...,639
8,E38-C,TGTGAAATCTTTGAAAACGAGATCGACTGGGATGAAATTGCCAGCA...,AATG,GCTT,CGTCTCGGTCTCAAATG,GCTTTGAGACCTGAGACG,CGTCTCGGTCTCAAATGTGTGAAATCTTTGAAAACGAGATCGACTG...,393
9,P42,CAGTACAAAATTTTTTAGATGCGTTCGCTGTCGATCCGGCCCGTCG...,GGAG,TACT,CGTCTCGGTCTCAGGAG,TACTTGAGACCTGAGACG,CGTCTCGGTCTCAGGAGCAGTACAAAATTTTTTAGATGCGTTCGCT...,133


In [93]:
fragments[['name', 'ext_sequence']].to_csv('datasets/jump/fragments-output.csv')

In [94]:
def design_primers(parts, prefix, suffix):
    
    primers_list = []
    for _, part in tqdm(parts.iterrows()):
        
        try:
            fp, rp = primers(part['sequence'], add_fwd=prefix + part['left_site'],
                             add_rev=str(Seq(suffix).reverse_complement()) + str(Seq(part['right_site']).reverse_complement()))
            
            #forward primers
            primers_list.append((part['name'], fp.seq, 
                                 fp.tm, fp.tm_total, fp.gc, fp.dg, fp.fwd, fp.offtargets, fp.penalty))
            #reverse primers
            primers_list.append((part['name'], rp.seq,
                                 rp.tm, rp.tm_total, rp.gc, rp.dg, rp.fwd, rp.offtargets, rp.penalty))
        
        except:
            print('Error on', part['name'], ': cannot find any feasible primer design, check your fragments!')
            continue
        
    return pd.DataFrame(primers_list, columns=['name', 'sequence', 'tm', 'tm_total', 'gc', 'dg', 'fwd', 'offtargets', 'penalty'])
        
raw_primers = design_primers(fragments, prefix, suffix)
#raw_primers

18it [00:06,  2.81it/s]


In [95]:
primer_seq = raw_primers[['fwd', 'sequence']].drop_duplicates().reset_index(drop=True)
primer_seq['primer_name'] = pd.Series(primer_seq.index).apply(lambda x: 'P' + str(x+1).zfill(len(str(primer_seq.shape[0]))))
primer_seq.loc[primer_seq['fwd'], 'primer_name'] = primer_seq['primer_name'] + '.F'
primer_seq.loc[~primer_seq['fwd'], 'primer_name'] = primer_seq['primer_name'] + '.R'
final_primers = pd.merge(primer_seq[['primer_name', 'sequence']], raw_primers.drop('name', axis=1), \
                         on='sequence', how='left').drop_duplicates().reset_index(drop=True)
final_primers

,primer_name,sequence,tm,tm_total,gc,dg,fwd,offtargets,penalty
0,P01.F,CGTCTCGGTCTCAGGAGGCCTCCACACCGCTCG,68.5,79.9,0.7,-1.768070,True,0,16.63614
1,P02.R,CGTCTCAGGTCTCAAGTAAGAGGTGTTACGATAGATGGGCG,68.2,75.2,0.5,-0.341245,False,0,7.88249
2,P03.F,CGTCTCGGTCTCAAATGATGAGCGATAGTCCGCA,62.8,74.1,0.5,-2.760765,True,0,11.32153
3,P04.R,CGTCTCAGGTCTCAAAGCTTATTATCTTCTCAGTACCTCTTCC,62.1,72.7,0.4,0.000000,False,0,3.40000
4,P05.F,CGTCTCGGTCTCAAATGACAGATGTAACGATTAAAAGAATAATATC,60.6,70.1,0.3,-3.571130,True,0,16.14226
5,P06.R,CGTCTCAGGTCTCAAAGCTTATTAGACGCTCGCG,60.5,74.2,0.5,-1.443465,False,0,10.38693
6,P07.F,CGTCTCGGTCTCAGGAGGCGCGGATAAAAATTTCATTTG,63.5,75.3,0.5,-3.634815,True,0,8.76963
7,P08.R,CGTCTCAGGTCTCAAGTATGGGTTCAACGATAGATGG,63.0,73.1,0.5,0.000000,False,0,4.00000
8,P09.F,CGTCTCGGTCTCAAATGAATGAAACCGATCCTGATC,60.0,72.2,0.5,-6.333900,True,0,17.66780
9,P10.R,CGTCTCAGGTCTCAAAGCTTATTATTCTTTAACATACAGGCAC,59.6,71.2,0.4,0.000000,False,0,5.70000


In [102]:
parts_x_primers = pd.merge(raw_primers[['name', 'sequence', 'fwd']], final_primers[['primer_name', 'sequence']], on='sequence', how='left')
parts_x_primers.shape

(36, 4)

In [ ]:
final_primers.to_csv('datasets/jump/final_primers.csv', index=False)

### Parts Amplification via Phusion PCR

#### Generate PCR Parameters

In [103]:
def get_annealing_part(a, b):
    return [a[i].replace(b[i], '') for i in range(len(b))]

def calculate_tm(seq):
    #return pr.calcTm(seq, dna_conc=250, dntp_conc=10,
    #                  tm_method='santalucia', salt_corrections_method='santalucia')
    
    return np.median([mt.Tm_Wallace(seq), mt.Tm_GC(seq), mt.Tm_NN(seq)])

pcr_primers = pd.merge(parts_x_primers, \
                fragments[['name', 'ext_sequence', 'left_overhang', 'right_overhang', 'size']], \
                on='name', how='left')
pcr_primers.loc[~pcr_primers['fwd'], 'sequence'] = pcr_primers['sequence']. \
            apply(lambda x: str(Seq(x).reverse_complement()))
pcr_primers.loc[pcr_primers['fwd'], 'overhang_part'] = pcr_primers['left_overhang']
pcr_primers.loc[~pcr_primers['fwd'], 'overhang_part'] = pcr_primers['right_overhang']
pcr_primers['annealing_part'] = get_annealed_part(pcr_primers['sequence'], pcr_primers['overhang_part'])
pcr_primers['tm'] = np.round(pcr_primers['annealing_part'].apply(calculate_tm), 1)
pcr_primers['tm_product'] = np.round(pcr_primers['ext_sequence'].apply(calculate_tm), 1)
pcr_primers['ext_time'] = pcr_primers['size'].apply(lambda x: max([60, x/1000 * 60]))
fwd_primers = pcr_primers[pcr_primers['fwd']] \
              [['name', 'primer_name', 'tm', 'tm_product', 'size', 'ext_time']]
rev_primers = pcr_primers[~pcr_primers['fwd']] \
              [['name', 'primer_name', 'tm', 'tm_product', 'size', 'ext_time']]

In [104]:
pcr_rxn = pd.merge(fwd_primers, rev_primers, on=['name', 'tm_product', 'size', 'ext_time'], how='outer')
pcr_rxn['ta'] = 0.3 * np.max([pcr_rxn['tm_x'], pcr_rxn['tm_y']], axis=0) + 0.7 * pcr_rxn['tm_product'] - 14.9
pcr_rxn['primer_name_x'] = pcr_rxn['primer_name_x'].str.split('_', expand=True)[0] \
                       .str.replace('(', '').str.replace(')', '')
pcr_rxn['primer_name_y'] = pcr_rxn['primer_name_y'].str.split('_', expand=True)[0] \
                       .str.replace('(', '').str.replace(')', '')
pcr_rxn = pcr_rxn[['name', 'size', 'primer_name_x', 'primer_name_y', 'ta', 'ext_time']] \
   .sort_values('ta').reset_index(drop=True)
pcr_rxn.columns = ['name', 'size', 'forward_primer', 'reversed_primer', 'annealing_temp', 'ext_time']
pcr_rxn = pd.merge(pcr_rxn, template, on='name', how='left')
pcr_rxn

,name,size,forward_primer,reversed_primer,annealing_temp,ext_time,template
0,RBS30,51,P24.F,P25.R,45.44,60,"3K3-araC-PBAD-30-gfp-B15, 3K3-araC-PBAD-(rJ)30..."
1,PCymRC,126,P28.F,P29.R,49.88,60,"3K3-cymR-PcymRC-(rJ)30-gfp-B15, 3K3-PcymRC-(rJ..."
2,PLuxB,91,P34.F,P35.R,51.98,60,3K3-PluxB-(rJ)30-gfp-B15
3,P20,96,P07.F,P08.R,54.44,60,3K3-P20*-gfp-B15-P(ara)-sECF20*-SP3-P(Cym)-32-...
4,PBAD,320,P26.F,P27.R,54.97,60,"3K3-araC-PBAD-30-gfp-B15, 3K3-araC-PBAD-(rJ)30..."
5,P38,116,P12.F,P13.R,55.10,60,3K3-P38*-gfp-B15-P(ara)-sECF38*-SP4-P(Cym)-32-...
6,E11-C,468,P05.F,P06.R,55.47,60,3K3-P11*-gfp-B15-P(ara)-sECF11*-SP11-P(Cym)-32...
7,E11-N,768,P03.F,P04.R,55.59,60,3K3-P11*-gfp-B15-P(ara)-sECF11*-SP11-P(Cym)-32...
8,E20-N,624,P09.F,P10.R,55.68,60,3K3-P20*-gfp-B15-P(ara)-sECF20*-SP3-P(Cym)-32-...
9,PSalTTC,152,P30.F,P31.R,55.69,60,3K3-PsalTTC-(rJ)30-gfp-B15


In [ ]:
pcr_rxn.to_csv('datasets/jump/pcr_rxn.csv', index=False)

In [105]:
valid_parts

,name,overhang,sequence,color,num_forbidden_sites
0,P11,P,GCCTCCACACCGCTCGTCACATCCTGTGATCCATTCCCCGCCCATC...,#f26520,0
1,E11-N,NOC,ATGAGCGATAGTCCGCAGAAACTGGGTCGTAATGAATGGAATGCAT...,#f26520,0
2,E11-C,NOC,ACAGATGTAACGATTAAAAGAATAATATCCAAAGGAGAACTTGAAT...,#f26520,0
3,P20,P,GCGCGGATAAAAATTTCATTTGCCCGCGACGGTTTTTCCGCCCATC...,#e23322,0
4,E20-N,NOC,AATGAAACCGATCCTGATCTGGAACTGCTGAAACGTATTGGTAATA...,#e23322,0
5,E20-C,NOC,CTGAAAAAAATCCTGAAAATCGAGGAACTGGATGAACGCGAACTGA...,#e23322,0
6,P38,P,CAGTACAAAATTTTTTAGATGCGTTGTACAACCCTCACGGGGGTGG...,#81171b,0
7,E38-N,NOC,CCTGCCGCACGTCCGGCACGTATTACCAATCAGCGTGATGGTGCAG...,#81171b,0
8,E38-C,NOC,TGTGAAATCTTTGAAAACGAGATCGACTGGGATGAAATTGCCAGCA...,#81171b,0
9,P42,P,CAGTACAAAATTTTTTAGATGCGTTCGCTGTCGATCCGGCCCGTCG...,#ca4642,0


#### Master Mix Calculation

In [ ]:
phusion_pcr = {
    'H2O': 12.4,
    'HF/GC Buffer': 4,
    'dNTPs': 0.4,
    #'FP': 1,
    #'RP': 1,
    #'DNA': 1,
    'DMSO': 0.6,
    'Phusion': 0.2,
    'Excess': 6
}

num_rxn = len(pcr_rxn)
master_mix = pd.DataFrame.from_dict(phusion_pcr, orient='index').rename(columns={0: 'vol'})[:-1]
master_mix['mix'] = master_mix['vol'] * num_rxn
factor = np.round((np.sum(master_mix)['mix'] + phusion_pcr['Excess'])/np.sum(master_mix)['vol'], 2)
master_mix['mix'] = master_mix['vol'] * factor
master_mix

#### Level 0 Assembly

In [ ]:
lvl0_rxn = {
    #'Fragment': 2,
    'UAC': 1,
    'BsmBI': 0.5,
    'T4-DNA-Ligase': 0.25,
    'Buffer': 2,
    'H2O': 14.25,
    'Excess': 6
}

num_rxn = len(pcr_rxn)
master_mix = pd.DataFrame.from_dict(lvl0_rxn, orient='index').rename(columns={0: 'vol'})[:-1]
master_mix['mix'] = master_mix['vol'] * num_rxn
factor = np.round((np.sum(master_mix)['mix'] + lvl0_rxn['Excess'])/np.sum(master_mix)['vol'], 2)
master_mix['mix'] = master_mix['vol'] * factor
master_mix